**Exercício 1** (0,75)

Modifique o código acima para fazer com que dois processos filhos alterem uma memória compartilhada contendo um inteiro. Faça o processo pai esperar por seus dois filhos para depois imprimir o valor da variável na tela. Dica: olhe o código do exercício 3 do Laboratório 1.

---

In [ ]:
%%writefile exercicio1.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main(int argc, char **argv)
{
  // Criando o segmento de memória compartilhada
  int segment_id = shmget(IPC_PRIVATE, sizeof(int), S_IRUSR | S_IWUSR);
  
  // Criação do ponteiro de memória compartilhada
  int *sharedMemory;

  // Criando o primeiro processo filho
  pid_t pid1 = fork();

  if (pid1 < 0) { // Erro na criação do processo
    printf("Sou o processo %d e nao consegui criar um novo processo.\n", getpid());
    return 1;
  } else if (pid1 == 0) { // PRIMEIRO PROCESSO FILHO
    // Anexando a memória compartilhada.
    sharedMemory = (int *)shmat(segment_id, NULL, 0);

    // Inicializando o valor na memória compartilhada
    *sharedMemory = 10;
    
    printf("Filho 1 (ID: %d) definiu o valor para: %d\n", getpid(), *sharedMemory);

    // Desanexando a memória compartilhada
    shmdt(sharedMemory);

    // Encerrando o processo filho
    exit(0);
  } else { // PROCESSO PAI
    // O pai cria o SEGUNDO processo filho
    pid_t pid2 = fork();

    if (pid2 < 0) { // Erro na criação do segundo processo
      printf("Sou o processo %d e nao consegui criar um segundo processo.\n", getpid());
      return 1;
    } else if (pid2 == 0) { // SEGUNDO PROCESSO FILHO
      // Anexando a memória compartilhada.
      sharedMemory = (int *)shmat(segment_id, NULL, 0);

      // O segundo filho modifica o valor já existente.
      // Ele adiciona 5 ao valor que o primeiro filho colocou (10).
      *sharedMemory += 5;

      printf("Filho 2 (ID: %d) somou 5. Novo valor: %d\n", getpid(), *sharedMemory);

      // Desanexando a memória compartilhada
      shmdt(sharedMemory);

      // Encerrando o processo filho
      exit(0);
    } else { // PROCESSO PAI NOVAMENTE
      wait(NULL);
      wait(NULL);

      // Anexando a memória compartilhada para ler o valor final.
      sharedMemory = (int *)shmat(segment_id, NULL, 0);

      // O printf agora exibe o valor final corretamente.
      printf("O valor final na memoria compartilhada é: %d\n", *sharedMemory);
      
      // Desanexando a memória
      shmdt(sharedMemory);

      // Removendo o segmento de memória compartilhada
      shmctl(segment_id, IPC_RMID, NULL);
    }
  }
  return 0;
}


Overwriting sharedMemory.c


In [ ]:
!gcc ./exercicio1.c -o exercicio1
!./exercicio1

Filho 1 (ID: 8640) definiu o valor para: 10
Filho 2 (ID: 8641) somou 5. Novo valor: 15
O valor final na memoria compartilhada é: 15


**Exercício 2** (0,75)

Utilizando comunicação entre processos, crie um código que escreva a Sequência de Fibonacci, com o parâmetro n sendo uma variável global. Tal sequência deve ser gerada por um processo filho, mas deve ser impressa na tela pelo processo pai. Dica: olhe o código do desafio do Laboratório 1.


---

In [5]:
%%writefile exercicio2.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>

int n = 20; // quantidade de termos da sequência de Fibonacci

int main(int argc, char **argv)
{
    int fd[2];
    if (pipe(fd) == -1) {
        perror("Erro ao criar pipe");
        exit(1);
    }

    pid_t pid = fork();

    if (pid == -1) {
        perror("Erro no fork");
        exit(1);
    }

    if (pid == 0) {
        // Processo filho -> gera Fibonacci e envia pelo pipe
        close(fd[0]); // fecha leitura (não vai usar)

        int a = 0, b = 1, c;
        write(fd[1], &a, sizeof(int)); // escreve primeiro termo
        if (n > 0) {
            write(fd[1], &b, sizeof(int)); // escreve segundo termo
        }

        for (int i = 2; i <= n; i++) {
            c = a + b;
            write(fd[1], &c, sizeof(int));
            a = b;
            b = c;
        }

        close(fd[1]); // fecha escrita
        exit(0);
    } else {
        // Processo pai -> lê do pipe e imprime
        close(fd[1]); // fecha escrita

        printf("Sequência de Fibonacci até n=%d:\n", n);
        int valor;
        while (read(fd[0], &valor, sizeof(int)) > 0) {
            printf("%d ", valor);
        }
        printf("\n");

        close(fd[0]); // fecha leitura
        wait(NULL);   // espera filho terminar
    }

    return 0;
}

Writing exercicio2.c


In [6]:
!gcc ./exercicio2.c -o exercicio2
!./exercicio2

Sequência de Fibonacci até n=20:
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 


**Exercício 3** (1,00)

Usando comunicação entre processos, escreva um código que realiza a soma de *n* números naturais, sendo *n* uma variável global. Cada processo deve fazer apenas a soma de dois números e o resultado final deve ser retornado pelo primeiro processo. Por exemplo, o processo 1 deve fazer a soma de 0+1, o processo 2 vai pegar esse resultado e somar com 2, e assim por diante.


---


In [3]:
%%writefile exercicio3.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>

int n = 10; // soma dos 10 primeiros números naturais (0+1+...+10)

int main(int argc, char **argv)
{
    int fd[2]; // pipe para comunicação
    if (pipe(fd) == -1) {
        perror("Erro ao criar pipe");
        exit(1);
    }

    pid_t pid;
    int acumulado = 0;

    // Processo pai inicial escreve o valor inicial (0) no pipe
    if (write(fd[1], &acumulado, sizeof(int)) == -1) {
        perror("Erro ao escrever no pipe");
        exit(1);
    }

    // Criar n processos (um para cada soma)
    for (int i = 1; i <= n; i++) {
        pid = fork();

        if (pid == -1) {
            perror("Erro no fork");
            exit(1);
        }

        if (pid == 0) { // processo filho
            int valor;
            // Lê o valor atual do pipe
            if (read(fd[0], &valor, sizeof(int)) == -1) {
                perror("Erro ao ler do pipe");
                exit(1);
            }

            // Soma o índice i
            valor += i;

            // Escreve de volta para o pipe
            if (write(fd[1], &valor, sizeof(int)) == -1) {
                perror("Erro ao escrever no pipe");
                exit(1);
            }

            exit(0); // filho encerra
        } else { // pai
            wait(NULL); // espera o filho terminar
        }
    }

    // No final, o processo pai pega o valor acumulado final
    if (read(fd[0], &acumulado, sizeof(int)) == -1) {
        perror("Erro ao ler resultado final");
        exit(1);
    }

    printf("Soma de 0 até %d = %d\n", n, acumulado);
    return 0;
}

Overwriting exercicio3.c


In [4]:
!gcc ./exercicio3.c -o exercicio3
!./exercicio3

Soma de 0 até 10 = 55
